<a href="https://colab.research.google.com/github/kyaoNK/bert_asag/blob/main/BERT_ASAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
!sudo apt -y update
# !sudo apt -y -q install libeigen3-dev libprotobuf-dev protobuf-c-compiler

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [71.4 kB]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:12 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,448 kB]
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:14 http://archive.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
import os
sys.path.append('/content/drive/MyDrive/Colab Notebooks/my-modules')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [4]:
!pip install -qU pytorch-lightning lightning-bolts 
!pip install -q   tqdm
!pip install -qU torch
!pip install -qU torchtext
!pip install -qU torchvision
!pip install -qU torchaudio
!pip install -q   datasets
!pip install -q   accelerate
!pip install -q   fugashi
!pip install -q   ipadic
!pip install -q   unidic-lite
!pip install -q   tqdm
!pip install -qU transformers
!pip install -q   sentencepiece
!pip install -U   sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 KB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.0/330.0 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 KB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.9/615.9 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 70.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 17.7 MB/s

# Datasets

In [5]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [6]:
# Constant
BATHC_SIZE = 32
# MODEL_CKPT = 'cl-tohoku/bert-base-japanese'

In [7]:
import pandas as pd

DATASET_DIR = '/content/drive/MyDrive/Colab Notebooks/SeniorThesis/DATASETS/OUTPUT/'

datafiles = {
    'jglue_train' : DATASET_DIR + 'dataset_jglue_train.csv',
    'jglue_test'  : DATASET_DIR + 'dataset_jglue_test.csv', 
    'jsick_train' : DATASET_DIR + 'dataset_jsick_train.csv',
    'jsick_test' : DATASET_DIR + 'dataset_jsick_test.csv', 
    'tutoring_tua_qr_train' : DATASET_DIR + 'dataset_tutoring_tua_train_qr.csv', 
    'tutoring_tua_qr_test'  : DATASET_DIR + 'dataset_tutoring_tua_test_qr.csv', 
    'tutoring_tua_ar_train' : DATASET_DIR + 'dataset_tutoring_tua_train_ar.csv', 
    'tutoring_tua_ar_test'  : DATASET_DIR + 'dataset_tutoring_tua_test_ar.csv',
}

# Fine-tuning

In [8]:
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import torch
import torch.optim as optim
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertModel
from transformers import BertJapaneseTokenizer
from torchmetrics.classification import F1Score

In [9]:
TEXT_COLUMNS = ['sentenceA', 'sentenceB']
LABEL_COLUMN = 'label_detail_asag'
# MODEL_CKPT = 'cl-tohoku/bert-base-japanese'
# MODEL_CKPT = 'cl-tohoku/bert-base-japanese-char-whole-word-masking'
MODEL_CKPT = "cl-tohoku/bert-base-japanese-whole-word-masking"

class CreateDataset(Dataset):
    def __init__(self, data, tokenizer, max_token_len):
        self.data = data
        self.tokenizer = tokenizer
        self.max_token_len = max_token_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        data_row = self.data.iloc[index]
        sentences = data_row[TEXT_COLUMNS]
        labels = data_row[LABEL_COLUMN]

        encodings = self.tokenizer(
            sentences[0], 
            sentences[1],
            add_special_tokens=True,
            truncation=True, 
            padding='max_length', 
            max_length=self.max_token_len,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return dict(
            # text=sentences.to_list(),
            input_ids=encodings['input_ids'].squeeze(1).flatten(),
            token_type_ids=encodings['token_type_ids'].squeeze(1).flatten(),
            attention_mask=encodings['attention_mask'].squeeze(1).flatten(),
            labels=torch.tensor(labels)
        )

class CreateDataModule(pl.LightningDataModule):
    def __init__(self, train_df, valid_df, test_df, batch_size=16, max_token_len=512,
                        pretrained_model=MODEL_CKPT):
        super().__init__()
        self.train_df = train_df
        self.valid_df = valid_df
        self.test_df = test_df
        self.batch_size = batch_size
        self.max_token_len = max_token_len
        self.tokenizer = BertJapaneseTokenizer.from_pretrained(pretrained_model)

    def setup(self, stage=None):
        self.train_dataset = CreateDataset(self.train_df, self.tokenizer, self.max_token_len)
        self.valid_dataset = CreateDataset(self.valid_df, self.tokenizer, self.max_token_len)
        self.test_dataset = CreateDataset(self.test_df, self.tokenizer, self.max_token_len)

    def train_dataloader(self):
        # return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, num_worker=8)
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.valid_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [10]:
# @title DATA
DATASET_NAMES = {'jglue':'a', 'jsick':'b'}
INPUT_NAMES = {'ar':'x','qr':'y'}
# COLUMNS = ['id', 'sentenceA', 'sentenceB', 'label']
def train_valid(df):
    X = df[TEXT_COLUMNS]
    y = df[LABEL_COLUMN]
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=0)
    train_df = X_train.merge( y_train , how='inner' , left_index=True , right_index=True )
    valid_df = X_valid.merge( y_valid , how='inner' ,left_index=True , right_index=True )
    return train_df , valid_df

def train_valid_test(pattern):
    if pattern[0] == 'a':   # jglue
        if pattern[1] == 'b':   # jglue-jsick
            if pattern[2] == 'x':      # *jglue-jsick-ar
                df_jglue_train, df_jglue_valid = train_valid(pd.read_csv(datafiles['jglue_train']))
                df_jsick_train, df_jsick_valid = train_valid(pd.read_csv(datafiles['jsick_train']))
                df_tua_train, df_tua_valid = train_valid(pd.read_csv(datafiles['tutoring_tua_ar_train']))
                train_df = pd.concat( [ df_jglue_train , df_jsick_train , df_tua_train ], axis=0)
                valid_df = pd.concat( [ df_jglue_valid , df_jsick_valid , df_tua_valid ], axis=0)
                test_df  = pd.concat( [ pd.read_csv(datafiles['jglue_test']) , pd.read_csv(datafiles['jsick_test']) , pd.read_csv(datafiles['tutoring_tua_ar_test']) ], axis=0)
            elif pattern[2] == 'y':    # *jglue-jsick-qr
                df_jglue_train, df_jglue_valid = train_valid(pd.read_csv(datafiles['jglue_train']))
                df_jsick_train, df_jsick_valid = train_valid(pd.read_csv(datafiles['jsick_train']))
                df_tua_train, df_tua_valid = train_valid(pd.read_csv(datafiles['tutoring_tua_qr_train']))
                train_df = pd.concat( [ df_jglue_train , df_jsick_train , df_tua_train ], axis=0)
                valid_df = pd.concat( [ df_jglue_valid , df_jsick_valid , df_tua_valid ], axis=0)
                test_df  = pd.concat( [ pd.read_csv(datafiles['jglue_test']) , pd.read_csv(datafiles['jsick_test']) , pd.read_csv(datafiles['tutoring_tua_qr_test']) ], axis=0)
        else :
            if   pattern[1] == 'x':     # *jglue-ar
                df_jglue_train, df_jglue_valid = train_valid(pd.read_csv(datafiles['jglue_train']))
                df_tua_train, df_tua_valid = train_valid(pd.read_csv(datafiles['tutoring_tua_ar_train']))
                train_df = pd.concat( [ df_jglue_train , df_tua_train ], axis=0)
                valid_df = pd.concat( [ df_jglue_valid , df_tua_valid ], axis=0)
                test_df  = pd.concat( [ pd.read_csv(datafiles['jglue_test']) , pd.read_csv(datafiles['tutoring_tua_ar_test']) ], axis=0)

            elif pattern[1] == 'y':    # *jglue-qr
                df_jglue_train, df_jglue_valid = train_valid(pd.read_csv(datafiles['jglue_train']))
                df_tua_train, df_tua_valid = train_valid(pd.read_csv(datafiles['tutoring_tua_qr_train']))
                train_df = pd.concat( [ df_jglue_train , df_tua_train ], axis=0)
                valid_df = pd.concat( [ df_jglue_valid , df_tua_valid ], axis=0)
                test_df  = pd.concat( [ pd.read_csv(datafiles['jglue_test']) , pd.read_csv(datafiles['tutoring_tua_qr_test']) ], axis=0)

    elif pattern[0] == 'b':  # jsick
        if   pattern[1] == 'x':    # *jsick-ar
            df_jsick_train, df_jsick_valid = train_valid(pd.read_csv(datafiles['jsick_train']))
            df_tua_train, df_tua_valid = train_valid(pd.read_csv(datafiles['tutoring_tua_ar_train']))
            train_df = pd.concat( [ df_jsick_train , df_tua_train ], axis=0)
            valid_df = pd.concat( [ df_jsick_valid , df_tua_valid ], axis=0)
            test_df  = pd.concat( [ pd.read_csv(datafiles['jsick_test']) , pd.read_csv(datafiles['tutoring_tua_ar_test']) ], axis=0)

        elif pattern[1] == 'y':    # *jsick-qr
            df_jsick_train, df_jsick_valid = train_valid(pd.read_csv(datafiles['jsick_train']))
            df_tua_train, df_tua_valid = train_valid(pd.read_csv(datafiles['tutoring_tua_qr_train']))
            train_df = pd.concat( [ df_jsick_train , df_tua_train ], axis=0)
            valid_df = pd.concat( [ df_jsick_valid , df_tua_valid ], axis=0)
            test_df  = pd.concat( [ pd.read_csv(datafiles['jsick_test']) , pd.read_csv(datafiles['tutoring_tua_qr_test']) ], axis=0)

    if   pattern[0] == 'x':         # *ar
        train_df, valid_df = train_valid(pd.read_csv(datafiles['tutoring_tua_ar_train']))
        # print(train_df)
        test_df  = pd.read_csv(datafiles['tutoring_tua_ar_test'])
        
    elif pattern[0] == 'y':        # *qr
        train_df, valid_df = train_valid(pd.read_csv(datafiles['tutoring_tua_qr_train']))
        test_df  = pd.read_csv(datafiles['tutoring_tua_qr_test'])

    test_df = test_df.drop('Unnamed: 0', axis=1)

    return train_df, valid_df, test_df

train_df, valid_df, test_df = train_valid_test( INPUT_NAMES['ar'] )

In [11]:
train_df

,sentenceA,sentenceB,label_detail_asag
1,問題とはあるべき理想の姿と現実とのギャップのことである。また、解決や解消を必要とする状況その...,状況に応じて発表方法を使い分けるなど， 効果的なフィードバックにつながるようにしないといけない,1
289,プリンタのLANポートにケーブルが接続されているか、無線LANなら適切なアクセスポイントに接...,"プリンタの電源が付いているか, インクなどがきれていないか,",0
8,コンピュータに入り込んで不正行為を行う悪意のあるプログラムのことである,コンピュータに入りこんで不正行為を行う悪意のあるプロ グラムのこと,6
214,数学や物理の知識が必要である,数学や物理の知識,6
101,コンピュータやサービスなどに他人のIDやパスワードを使ってアクセスする不正アクセス、ホームペ...,サイバー犯罪は「不正アクセス禁止法違反」、「コンピュータ・電磁的記録対象犯罪等」、「ネットワ...,5
...,...,...,...
323,プログラミング言語とは違い、人間の日常の会話や文書で用いる日本語や英語などの言語である,人間に日常で使用される日本語や英語などの言語,5
192,時間的要素を含まないモデルを静的モデル、時間的要素を含むものを動的モデルという,静的モデル，動的モデル,5
117,デジタル画像において画素という点の集まり具合を解像度という,画素（ピクセル）の集まり具合を解像度といい、ラスタデータにおいて画像の細かさを示すもので、1...,6
47,バリアフリーは障がい者や高齢者、妊婦にとって障壁となるものを取り除くという考えで、ユニバーサ...,バリアフリーは誰かにとって障壁となっているものを取り除こうという考え方、ユニバーサルデザイン...,2


In [12]:
valid_df

,sentenceA,sentenceB,label_detail_asag
122,メタファーとは違う分野の言葉を喩えに使って表現する方法であり、特に目には見えないものを伝える...,よくわからない物事を、すでに知っている似たことに例えること,2
66,問題解決の考え方を適応し、問題点を明確に、具体的な理想を明らかにして分析を進め、情報や情報技...,問題点を明確にして具体的な理想を明らかにして分析を進めるとともに情報や情報技術を効果的に利用...,5
142,情報機器の登場により、様々なデータが大量に通信できるようになり、双方向でのコミュニケーション...,様々なデータを大量に通信することと、双方向でのコミュニケーション。,2
246,データを可視化して問題発見や問題解決を行う,，データを可視化することによる問題発見や問題解決,6
146,モダンデザインとは装飾によるデザインではなく、製品が持つ本質的な機能美を追求した合理的・機能...,装飾によるデザインから製品が持つ本質的な機能美を追求することで工業製品に取り入れることができ...,3
...,...,...,...
235,数学や物理の知識が必要である,数学や物理の知識が必要,6
74,アイデアの近さやまとまりを可視化するKJ法や、一定のルールに基づいて構造的にアイデアを表すも...,KJ法・ロジックツリー・コンセプトマップ,5
52,テキストや文書ファイルは完全に復元する必要があるため可逆圧縮を用いる。動画や音声、画像ファイ...,テキストや文書ファイルは閲覧する際に圧縮を完全に戻す必要があるため可逆圧縮を用いるが、画像や...,6
236,ブラウン運動や株価の変動など不確実な現象のシミュレーションに用いられる,ブラウン運動や株価の変動など不確実な現象のシミュレーションに用いられる,6


In [13]:
test_df

,sentenceA,sentenceB,label_detail_asag
0,データは事象や現象を数字や文字で記号化したもので、情報は意味や価値が付加されたデータのことである,事象や現象を数字や文字などで記号化したものがデータであり、意味や価値が付加されたデータやメッ...,6
1,情報の特性は形がない、消えない、複製できる、容易に伝播することである,①形がない ②消えない ③簡単に複製できる ④容易に伝播する,6
2,問題とはあるべき理想の姿と現実とのギャップのことである。また、解決や解消を必要とする状況その...,理想と現実の間にあるギャップ,2
3,ブレーンストーミングのルールは質より量、制約を設けない、便乗の推奨、批判の禁止である,①アイデア量を置く出す②制約を無くして寛容的に行う③他者の意見に乗りさらに広げる④批判をしない,4
4,アイデアの近さやまとまりを可視化するKJ法や、一定のルールに基づいて構造的にアイデアを表すも...,KJ法、コンセプトマップ、ロジックツリー、座標軸等,5
...,...,...,...
115,名義尺度と順序尺度をまとめてカテゴリ型データという,名義尺度と順序尺度のデータ.,2
116,分割表やマリメッコチャートを用いる,"分割表やモザイク図, マリメッコチャートなどを用いて表す.",6
117,統計解析ソフトウェアであるRやTableau社が提供しているTableauDesktopを用いる,ソフトウェアRなど,4
118,数値で表現されていても表現されていても量的データと同様の演算や可視化を行うことができない,量的データに適用できる可視化手法や演算をそのまま利用できない点,3


In [14]:
data_module = CreateDataModule(train_df, valid_df, test_df)
data_module.setup()

In [19]:
data_module

In [25]:
class Model(pl.LightningModule):
    def __init__(self, n_classes:int, n_epochs=None, pretrained_model=MODEL_CKPT):
        super().__init__()

        self.bert = BertModel.from_pretrained(pretrained_model, return_dict=True)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)
        self.n_epochs = n_epochs
        self.criterion = nn.CrossEntropyLoss()
        self.f1_score = F1Score(task='multiclass', num_classes=n_classes)

        for param in self.bert.parameters():
            param.requires_grad = False
        for param in self.bert.encoder.layer[-1].parameters():
            param.requires_grad = True

    def forward(self, input_ids, token_type_ids, attention_mask, labels=None):
        # print(input_ids.size())
        # print(token_type_ids.size())
        # print(attention_mask.size())

        output = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        preds = self.classifier(output.pooler_output)
        loss = 0
        if labels is not None:
            loss = self.criterion(preds, labels)
        return loss, preds

    def training_step(self, batch, batch_idx):
        loss, preds = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['labels']
        )
        return {'loss' : loss, 'batch_preds' : preds, 'batch_labels' : batch['labels']}

    def validation_step(self, batch, batch_idx):
        loss, preds = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['labels']
        )
        return {'loss' : loss, 'batch_preds' : preds, 'batch_labels' : batch['labels']}

    def test_step(self, batch, batch_idx):
        loss, preds = self.forward(
            input_ids=batch['input_ids'],
            attention_mask=batch['attention_mask'],
            token_type_ids=batch['token_type_ids'],
            labels=batch['labels']
        )
        return {'loss' : loss, 'batch_preds' : preds, 'batch_labels' : batch['labels']}

    def validation_epoch_end(self, outputs, mode='val'):
        epoch_preds = torch.cat([x['batch_preds'] for x in outputs])
        epoch_labels = torch.cat([x['batch_labels'] for x in outputs])
        epoch_loss = self.criterion(epoch_preds, epoch_labels)
        self.log(f'{mode}_loss', epoch_loss, logger=True)

        num_correct = (epoch_preds.argmax(dim=1) == epoch_labels).sum().item()
        epoch_accuracy = num_correct / len(epoch_labels)
        self.log(f'{mode}_accuracy', epoch_accuracy, logger=True)

        epoch_f1_score = self.f1_score(epoch_preds, epoch_labels)
        self.log(f'{mode}_f1_score', epoch_f1_score, logger=True)

        return 

    def test_epoch_end(self, outputs):
        # return self.validation_epoch_end(outputs, 'test')
        self.validation_epoch_end(outputs, 'test')

    def configure_optimizers(self):
        optimizer = optim.Adam([
            {'params' : self.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
            {'params' : self.classifier.parameters(), 'lr': 1e-4}
        ])

        return [optimizer]

In [26]:
N_EPOCHS = 3

model = Model(n_classes=7, n_epochs=N_EPOCHS)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    min_delta=0.01,
    patience=3,
    mode='min'
)

loss_checkpoint = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='best_loss_{epoch}',
    verbose=True,
    monitor='val_loss',
    save_weights_only=True,
    mode='min'
)

acc_checkpoint = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='best_acc_{epoch}',
    verbose=True,
    monitor='val_accuracy',
    save_weights_only=True,
    mode='max'
)

f1_checkpoint = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='best_f1_{epoch}',
    verbose=True,
    monitor='val_f1_score',
    save_weights_only=True,
    mode='max'
)

trainer = pl.Trainer(
    max_epochs=N_EPOCHS,
    gpus=1,
    # progress_bar_refresh_rate=30,
    callbacks=[early_stop_callback, loss_checkpoint, acc_checkpoint, f1_checkpoint]
    # callbacks=[early_stop_callback, loss_checkpoint]
)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:467: LightningDeprecatio

In [27]:
trainer.fit(model, data_module)
# trainer.fit(model, dataloaders)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /content/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type              | Params
-------------------------------------------------
0 | bert       | BertModel         | 110 M 
1 | classifier | Linear            | 5.4 K 
2 | criterion  | CrossEntropyLoss  | 0     
3 | f1_score   | MulticlassF1Score | 0     
-------------------------------------------------
7.1 M     Trainable params
103 M     Non-trainable params
110 M     Total params
442.491   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/trainer.py:1600: PossibleUserWarning: The number of training batches (17) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 17: 'val_loss' reached 1.52616 (best 1.52616), saving model to '/content/checkpoints/best_loss_epoch=0.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 17: 'val_accuracy' reached 0.40909 (best 0.40909), saving model to '/content/checkpoints/best_acc_epoch=0.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 0, global step 17: 'val_f1_score' reached 0.40909 (best 0.40909), saving model to '/content/checkpoints/best_f1_epoch=0.ckpt' as top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 34: 'val_loss' reached 1.48311 (best 1.48311), saving model to '/content/checkpoints/best_loss_epoch=1.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 34: 'val_accuracy' was not in top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 1, global step 34: 'val_f1_score' was not in top 1


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 51: 'val_loss' reached 1.43318 (best 1.43318), saving model to '/content/checkpoints/best_loss_epoch=2-v2.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 51: 'val_accuracy' reached 0.48485 (best 0.48485), saving model to '/content/checkpoints/best_acc_epoch=2-v2.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:Epoch 2, global step 51: 'val_f1_score' reached 0.48485 (best 0.48485), saving model to '/content/checkpoints/best_f1_epoch=2-v2.ckpt' as top 1
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


In [28]:
result = trainer.test(ckpt_path=loss_checkpoint.best_model_path)
result = trainer.test(ckpt_path=acc_checkpoint.best_model_path)
result = trainer.test(ckpt_path=f1_checkpoint.best_model_path)

ValueError: ignored

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /content/lightning_logs